# task 
- buat dataset baru merubah var jadi sub var
- buat fungsi dengan lebih efisien
- simpan ke netcdf

In [1]:
import pandas as pd
import numpy as np
import xarray
from numba import njit, jit

In [2]:
# ### States
path_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/states_new.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="netcdf4")

In [3]:
crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True)

fao_stat = pd.read_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Production_Crops_Livestock_E_All_Data_analyze.csv")
fao_stat_area_harv = fao_stat[fao_stat['Element'] == "Area harvested"]
fao_stat_area_harv.insert(9, 'True Value', fao_stat_area_harv['Value']/100, True)
faostat_area_harvested = fao_stat_area_harv.merge(crop_class, on='Item')

In [4]:
gfrac_interp = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GFRAC-INTERPOLATION-WORLDWIDE-1970_1975.nc", engine="netcdf4")
luc_5cat = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION_5CAT_5thn_new.nc", engine="netcdf4")

static = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/staticData_quarterdeg.nc'
luh2_static = xarray.open_dataset(static, engine="netcdf4")

staticcode = 'D:/kerja/asisten riset/code v/gpw_v4_national_identifier_grid_rev11_15_min_finall.nc'
luh2_static2 = xarray.open_dataset(staticcode, engine="netcdf4")

luh2_merge = xarray.merge([luh2_static2, luh2_static], join='right', compat="override")

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code.xlsx", engine="openpyxl")
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['name'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]
    
ccode_worldwide_int = luh2_merge['ccode'].to_numpy().astype('int64')
ccode_convert = np.zeros((720, 1440), dtype="<U64")

for i in range(720):
    for j in range(1440):
        if ccode_worldwide_int[i][j] in ccode_dict.keys():
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"

country = xarray.Dataset({
    "country": (["lat", "lon"], ccode_convert)
    },coords={
        "lon":luh2_states.coords["lon"].to_numpy(),
        "lat":luh2_states.coords["lat"].to_numpy()
})

In [250]:

def grid_check_process(time, lat, lon, luh2_cat, ngfbfc_att, country):
    year = pd.to_datetime(luh2_states['time'].isel(time=time).values, format='%d/%m/%Y %H.%M.%S').year
    
    crops = []
    jenis_kacang_kacangan = []
    crops_sum = 0
    emission_intensity = []
    major_crops = []

    for luh2 in luh2_cat:
        for ngfbfc in ngfbfc_att:
            query = (faostat_area_harvested['Area'] == country) & \
                    (faostat_area_harvested['Year'] == year) & \
                    (faostat_area_harvested['IMAGE Classification'] == ngfbfc) & \
                    (faostat_area_harvested['LUH2Class'] == luh2)
            
            jenis_kacang_kacangan += faostat_area_harvested[query]['Item'].tolist()
            crops += faostat_area_harvested[query]['True Value'].tolist()
            crops_sum += faostat_area_harvested[query]['True Value'].sum()


            luc_5cat_val = luc_5cat['luc_crops'].isel(time=time, lat=lat, lon=lon).sel(cat=luh2_cat).data.tolist()
            gfrac_interp_val = gfrac_interp['GFRAC_interp'].isel(time=time, lat=lat, lon=lon).sel(NGFBFC=ngfbfc_att).data.tolist()
            carea_val = luh2_static['carea'].isel(lat=lat, lon=lon).data.tolist()
            
            proportion = (crops / crops_sum) * 100

            # kondisi A 
            if len(luh2_cat) == 1 and len(ngfbfc_att) == 1:
                emission_intensity += luc_5cat_val / (gfrac_interp_val * carea_val)

            # Kondisi B 
            elif len(luh2_cat) == 1 and len(ngfbfc_att) > 1:
                luh2_item = faostat_area_harvested[query]['LUH2Class'].tolist()
                if luh2_cat == luh2_item:
                    emission_intensity += luc_5cat_val / (gfrac_interp_val * carea_val)
                else:
                    emission_intensity = 0

            #Kondisi C
            elif len(luh2_cat) > 1 and len(ngfbfc_att) == 1:
                crops_carea = proportion * np.multiply(gfrac_interp_val, carea_val) / 100
                if len(crops_carea) != len(luc_5cat_val):
                    crops_carea = np.repeat(crops_carea, len(luc_5cat_val) // len(crops_carea) + 1)[:len(luc_5cat_val)]
                try:
                    intensity = luc_5cat_val / crops_carea
                except ZeroDivisionError:
                    intensity = np.zeros_like(luc_5cat_val)
                    
                emission_intensity += intensity.tolist()

            #kondisi D1 D2
            elif len(luh2_cat) > 1 and len(ngfbfc_att) > 1:
                grid_major = np.array(['Oil, palm fruit', 'Wheat', 'Soybeans', 'Maize', 'Rice'])
                gfrac_interp_major = gfrac_interp['GFRAC_interp'].isel(time=time, lat=lat, lon=lon)['NGFBFC']
                list_major_found = gfrac_interp_major[np.isin(gfrac_interp_major, grid_major)].data
                major_crops = gfrac_interp['GFRAC_interp'].isel(time=time, lat=lat, lon=lon).sel(NGFBFC=list_major_found).values * carea_val

                agri_total = sum(gfrac_interp_val) * carea_val
                non_major_crops = agri_total - sum(major_crops)
                crops_carea = (proportion * non_major_crops) / 100

                # Sesuaikan bentuk array crops_carea jika diperlukan
                if len(crops_carea) != len(luc_5cat_val):
                    crops_carea = np.repeat(crops_carea, len(luc_5cat_val) // len(crops_carea) + 1)[:len(luc_5cat_val)]

                try:
                    intensity = luc_5cat_val / crops_carea
                except ZeroDivisionError:
                    intensity = np.zeros_like(luc_5cat_val)
                emission_intensity += intensity.tolist()

    return jenis_kacang_kacangan, emission_intensity

In [251]:
def check_attribut(time, lat, lon):
    country = ccode_convert[lat][lon]
    
    grid_gfrac = gfrac_interp['GFRAC_interp'].isel(time=time, lat=lat, lon=lon) #90
    states = luh2_states['luh2_states'].isel(time=time, lat=lat, lon=lon) #80

    att_ngfbfc = gfrac_interp.coords['NGFBFC'].where(grid_gfrac > 0.0, drop=True).data.tolist()
    att_states = luh2_states.coords['cat'].where(states > 0.0, drop=True).data.tolist()
    
    if att_ngfbfc != att_states:
        return grid_check_process(time, lat, lon, att_states, att_ngfbfc, country) 
    else:
        return 0

In [252]:
# lat_indices, lon_indices = np.where(ccode_convert == "Indonesia")

ds = xarray.Dataset(
    coords={
        'time': pd.date_range(start="1970-01-01", end="1971-01-01", freq='YS'),
        'lat': luh2_states.coords["lat"].to_numpy(),
        'lon': luh2_states.coords["lon"].to_numpy(),
        "country": country['country']
    }
)

In [253]:
for time in range(2):
    for lat in range(720):
        for lon in range(420):
            data = check_attribut(time, lat, lon)
            if data != 0:    
                jenis_kacang_kacangan = data[0]
                emisi_intensitas = data[1]

                for jenis, intensitas in zip(jenis_kacang_kacangan, emisi_intensitas):
                    ds[jenis] = xarray.DataArray(
                        np.zeros((len(ds['time']), len(ds['lat']), len(ds['lon']))),
                        dims=("time", "lat", "lon")
                    )
                    ds[jenis][:, lat, lon] = intensitas
            else:
                continue

ZeroDivisionError: integer division or modulo by zero

In [ ]:
# import xarray as xr
# import numpy as np

# # Dapatkan nama-nama jenis kacang-kacangan dan hasil perhitungan emisi intensitas dari fungsi grid_check_process
# jenis_kacang_kacangan = data[0]
# emisi_intensitas = np.zeros(len(jenis_kacang_kacangan))

# for i, emisi in enumerate(data[1][:36]):
#     if emisi_intensitas[i]:
#         emisi_intensitas[i] = emisi
#     else: 
#         emisi_intensitas[i] = 0 

# time = pd.to_datetime(["1971-01-01"])
# lat = luh2_states.coords["lat"].to_numpy()
# lon = luh2_states.coords["lon"].to_numpy()

# ds = xr.Dataset(
#     coords={
#         'time': pd.to_datetime(["1971-01-01"]),
#         'lat': luh2_states.coords["lat"].to_numpy(),
#         'lon': luh2_states.coords["lon"].to_numpy(),
#         "country": country['country']
#     }
# )

# # Buat variabel dalam netCDF untuk menyimpan data emisi intensitas untuk setiap jenis kacang-kacangan
# for jenis in jenis_kacang_kacangan:
#     ds[jenis] = xr.DataArray(np.zeros((len(time), len(lat), len(lon))), dims=("time", "lat", "lon"))

# # Tetapkan nilai-nilai variabel berdasarkan hasil perhitungan emisi intensitas
# for i, jenis in enumerate(jenis_kacang_kacangan):
#     ds[jenis][:, :, :] = emisi_intensitas[i]  # Tetapkan nilai emisi intensitas untuk setiap jenis kacang-kacangan

# # ds.to_netcdf('emisi_intensitas_kacang_kacangan.nc')
# ds